In [57]:
# Tarvail réalisé par Salwa AZARIOUH et Alexandre MARTEL


from mesa import Agent, Model
from mesa.datacollection import DataCollector
from mesa.time import RandomActivation, SimultaneousActivation
from mesa.space import MultiGrid
import random
import numpy
from enum import IntEnum

class State(IntEnum):
    SUSCEPTIBLE = 0 #susceptible d'être infecté
    EXPOSED = 1
    INFECTED_LOW = 2 # Benin
    INFECTED_HARD = 3 # Grave
    RECOVERED = 4 #Ne peut être infecté

class Background(IntEnum):
    PROLETAIRE = 0
    MOYEN = 1
    BOURGEOIS = 2 
    VILLAGEOIS = 3
    
class Genre(IntEnum):
    GARCON = 0
    FILLE = 1
    HOMME = 2
    FEMME = 3


class Person(Agent):
    def __init__(self, unique_id, model, background, health_state=State.SUSCEPTIBLE, water_source=None, ):
        super().__init__(unique_id, model)
        self.health_state = health_state  
        self.water_source = water_source  # Point d'eau où l'agent se rend
        self.genre = random.choices([Genre.GARCON, Genre.FILLE, Genre.HOMME, Genre.FEMME], [0.15,0.15,0.35,0.35])  # Assignation d'un âge aléatoire
        self.time_infected = 0  # Durée depuis laquelle l'agent est infecté
        self.background = background  #Classe sociale de l'individu
        self.time_recovered = 0 # Durée depuis laquelle l'agent est guérit
    
    def recover_or_die_hard(self):
        
        if self.random.random() < 0.05:  # 5% de chance de mourir
            self.model.grid.remove_agent(self)
            self.model.schedule.remove_agent(self)
        else: 
            self.health_state = State.RECOVERED  # Guéri
            self.time_infected = 0
            
    def recover_or_die_low(self):
        
        self.health_state = State.RECOVERED  # Guéri
        self.time_infected = 0

    def recovered(self):
        if self.time_recovered > 240:
            self.health_state = State.SUSCEPTIBLE
            self.time_recovered = 0
            
    def drink_water(self):
        if self.water_source and self.water_source.contaminated:
            if self.health_state == State.SUSCEPTIBLE and self.random.random() < 0.3:  # 30% de chance d'être exposed
                self.health_state = State.EXPOSED  # Passe de Susceptible à Exposed
        elif self.water_source and not(self.water_source.contaminated):
            if self.health_state == State.EXPOSED:
                self.water_source.contaminated = True
                
    def contact(self):
        #je vérifie s'il y a d'autres agents avec moi dans la même case
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        
        #si je ne suis pas tout seul et je suis infecté
        if len(cellmates)>1 and (self.health_state == State.INFECTED_LOW or self.health_state == State.INFECTED_HARD):
            
            # self.model.ptrans attribut à définir au niveau du modèle
            for other in cellmates:
                if other.health_state == State.SUSCEPTIBLE and self.random.random() < self.model.ptrans :
                    other.health_state = State.EXPOSED
    
    def move(self):
        #récupérer la liste des positions autour de la case là ou la cellule se trouve
        possible_steps = self.model.grid.get_neighborhood(self.pos,moore=True, include_center=True)
        
        new_position = self.random.choice(possible_steps)
        
        self.model.grid.move_agent(self, new_position) #l'attribut self.pos va changer automatiquement
        
    def status(self):
        if self.health_state == State.EXPOSED:
            self.time_infected += 1
            if (self.time_infected >= 2 and random.randint(0,2) == 2) or self.time_infected == 5:  
                
                self.health_state = random.choices([State.INFECTED_HARD, State.INFECTED_LOW], [0.1, 0.9])
                
        elif self.health_state == State.INFECTED_HARD and self.time_infected >= 5:
            self.recover_or_die_hard()
        
        elif self.health_state == State.INFECTED_LOW and self.time_infected >= 10:
            self.recover_or_die_low()
            
        elif self.health_state == State.RECOVERED:
            self.time_recovered += 1
            self.recovered()
        
    def step(self):
        
        self.move()
        self.contact()
        self.status()
    
            
class CholeraModel(Model):
    def __init__(self, width, height, population, ptrans):
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        
        self.datacollector = DataCollector(
            agent_reporters={
                "State": "health_state"  # Nom de la variable d'état des agents
            }
        )

        # Initialiser les agents humains
        for number_type in range(4):
            
            if number_type == 0:
                background = Background.PROLETAIRE
            if number_type == 1:
                background = Background.VILLAGEOIS
            if number_type == 2:
                background = Background.MOYEN
            if number_type == 3:
                background = Background.BOURGEOIS
                
            for i in range(population[number_type]):
                person = Person(number_type*100 + i, self, background)
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height)
                self.grid.place_agent(person, (x, y))
                self.schedule.add(person)

        # Initialiser des sources d'eau et infrastructures
        for i in range(10):  # On rajoute 10 points d'eau
            water_source = WaterSource(i, self, contaminated=False)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(water_source, (x, y))
            
    def step(self):
        self.datacollector.collect(self)  # Collecte les données à chaque pas
        for agent in self.schedule.agents:
            if isinstance(agent, Person):
                if agent.health_state == State.INFECTED_HARD or agent.health_state == State.INFECTED_LOW:  # Si l'agent est infecté
                    agent.drink_water()  # Boit de l'eau, potentiellement contaminée
        self.schedule.step()  # Avance la simulation d'un pas

class WaterSource(Agent):
    def __init__(self, unique_id, model, contaminated=False):
        super().__init__(unique_id, model)
        self.contaminated = contaminated

    def step(self):
        pass


        
        


In [58]:
import pandas as pnd


def get_column_data(model):
    """pivot the model dataframe to get states count at each step"""
    agent_state = model.datacollector.get_agent_vars_dataframe()
    X = pnd.pivot_table(agent_state.reset_index(),index='Step',columns='State',aggfunc=numpy.size,fill_value=0)    
    labels = ['Susceptible','Infected','Removed']
    X.columns = labels[:len(X.columns)]
    return X



In [63]:
from bokeh.models import ColumnDataSource, Line, Legend
from bokeh.palettes import Category10
from bokeh.plotting import figure
from bokeh.io import push_notebook, output_notebook,output_file,show
from bokeh.models import HoverTool, LinearColorMapper
import numpy as np

def plot_states_bokeh(model,title=''):
    """Plot cases per country"""

    X = get_column_data(model)
    X = X.reset_index()
    source = ColumnDataSource(X)
    i=0
    colors = Category10[3]
    items=[]
    p = figure(width=600,height=400,tools=[],title=title,x_range=(0,100))        
    for c in X.columns[1:]:
        line = Line(x='Step',y=c, line_color=colors[i],line_width=3,line_alpha=.8,name=c)
        glyph = p.add_glyph(source, line)
        i+=1
        items.append((c,[glyph]))

    p.xaxis.axis_label = 'Step'
    p.add_layout(Legend(location='center_right',   
                items=items))
    p.background_fill_color = "#e1e1ea"
    p.background_fill_alpha = 0.5
    p.legend.label_text_font_size = "10pt"
    p.title.text_font_size = "15pt"
    p.toolbar.logo = None
    p.sizing_mode = 'scale_height'    
    return p


def plot_cells_bokeh(model):

    agent_counts = np.zeros((model.grid.width, model.grid.height))
    w=model.grid.width
    df=grid_values(model)
    df = pnd.DataFrame(df.stack(), columns=['value']).reset_index()    
    columns = ['value']
    x = [(i, "@%s" %i) for i in columns]    
    hover = HoverTool(
        tooltips=x, point_policy='follow_mouse')
    colors = Category10[3]
    mapper = LinearColorMapper(palette=colors, low=df.value.min(), high=df.value.max())
    p = figure(width=500,height=500, tools=[hover], x_range=(-1,w), y_range=(-1,w))
    p.rect(x="level_0", y="level_1", width=1, height=1,
       source=df,
       fill_color={'field':'value', 'transform': mapper},
       line_color='black')
    p.background_fill_color = "black"
    p.grid.grid_line_color = None    
    p.axis.axis_line_color = None
    p.toolbar.logo = None
    return p

def grid_values(model):
    """Get grid cell states"""

    agent_counts = np.zeros((model.grid.width, model.grid.height))
    w=model.grid.width
    df=pnd.DataFrame(agent_counts)
    for cell in model.grid.coord_iter():
        agents, (x, y) = cell
        c=None
        for a in agents:
            if isinstance(a, Person):  # Vérifie si l'agent est de type Person
                c = a.health_state  # Récupère l'état de santé de l'agent
                break  # On peut sortir de la boucle dès qu'on trouve un agent de type Person
        df.iloc[x,y] = c
    return df

In [64]:

steps=400
pop=400

output_notebook()

model = CholeraModel(20, 20, [30,15,20,5], ptrans=0.25)
for i in range(steps):
    model.step()
    p1=plot_states_bokeh(model,title='step=%s' %i)
    
    p2=plot_cells_bokeh(model) 
    
    show(p1)
    show(p2)

Loading BokehJS ...

c:\Users\alexa\Desktop\Projet_Info\mesa\.venv\Lib\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)


: 